Table of final selected target genes with all logFCs, pvalues and screening scores.

# Selected target genes

__Rho_down & Exp_down & Hyper_m6A__

> #### __SQLE__
> #### Value of SQLE Expression in Prognostic Evaluation of Patients with Acute Myeloid Leukemia 
https://pubmed.ncbi.nlm.nih.gov/33554790/

> #### __INTS5__
> #### Coordinated Alterations in RNA Splicing and Epigenetic Regulation Drive Leukemogenesis 
https://www.nature.com/articles/s41586-019-1618-0
>> Transcriptional pausing was also evident at INTS5 and INTS14 in SRSF2 mutant cells (Extended Data Fig. 7r, ​,s),s), which, in combination with aberrant splicing of several Integrator subunits (Supplementary Table 24), suggested impaired function of the entire Integrator complex in SRSF2 mutant cells

> #### __BCL2__
> #### Venetoclax combined with decitabine or azacitidine in treatment-naive, elderly patients with acute myeloid leukemia 
> #### Clinical Trials & Observations 
https://doi.org/10.1182/blood-2018-08-868752

> #### __MYBBP1A__
> #### Identification of Myb-binding Protein 1A (MYBBP1A) as a Novel Substrate for Aurora B Kinase 
> https://doi.org/10.1074/jbc.M109.068312

In [3]:
import sys 
import pandas as pd 
import numpy as np
from matplotlib.pyplot import subplots
from itertools import chain, product
from matplotlib_venn import venn3,venn3_circles, venn3_unweighted
from venn import venn 

import upsetplot as us
from matplotlib.backends.backend_pdf import PdfPages

sys.path.append('../')

from util import *

In [4]:
# wd = '/rumi/shams/abe/Projects/Decitabine-treatment/'
wd = '/data_gilbert/home/aarab/Projects/Decitabine-treatment'

## Load data
Comparing treated with Decitabine vs. non-treated:
- **$\Delta$RNA methylation**  (hl60 cell line)
- **$\Delta$Translational efficiency** (hl60 cell line)
- **$\Delta$RNA expression** (For all 6 AML cell lines)
- **$\Delta$RNA stability** (For all 6 AML cell lines)
- **$\Delta$Phenotype, CRISPRi-screen $\rho$(rho) score** (hl60 and molm13 cell lines) 

In [5]:
comps = load_data(comparisons=True,wd=wd)

In [6]:
scrns = load_data(screens=True,wd=wd)

## Generate final merged table

In [10]:
all_genes_list = [
    'SQLE',
    'INTS5',
    'BCL2',
    'MYBBP1A'
]

In [385]:
# all_genes = {}
# all_genes_list = []

# for i,j,k in iter_by_two(['up', 'down'],['up', 'down'],['Hyper', 'Hypo']):
    
#     for third in ['Exp','TE','Stbl']:
#         name = f'Rho_{i} & {k}_m6A & {third}_{j}'
#         genes = list(set(eval(f'Rho_{i}')).intersection(eval(f'{k}_m6A'),eval(f'{third}_{j}')))
#         all_genes[name] = genes
#         all_genes_list = all_genes_list + genes
        
# all_genes_list = set(all_genes_list)

In [8]:
def get_intersect_df(genes, data, key=None):
    # get intersects 
    if key is not None: 
        df = data[key]
    else: 
        df = data

    out = df.iloc[[i for i, g in enumerate (df.gene_name) if g in list(genes)],]

    return out


def make_comp_final_table(genes,data):
    E = get_intersect_df(genes, data['delta_exp'])
    S = get_intersect_df(genes, data['delta_stbl'])
    T = get_intersect_df(genes, data['delta_te'])

    out = pd.DataFrame(index=genes)

    out = pd.concat([
        out,
        # RNA Expression
        E.reset_index(drop=True).set_index('gene_name').add_prefix('Exp.'),
        # RNA Stability 
        S.reset_index(drop=True).set_index('gene_name').add_prefix('Stbl.'),
        # Translational Efficiency
        T.reset_index(drop=True).set_index('gene_name').add_prefix('TE.'),
    ],axis=1)

    out = out.round(5)
    return out

def make_scrn_final_table(genes,data):
    return merge_screen_data(
        cells= ['hl60','molm13'], 
        exps = ['exp1','exp2','exp'], 
        drugs=['DAC'], 
        scores = ['gamma','rho'], data=data
    ).loc[genes,:]

def make_mtyl_final_table(genes):
    mtyl_bed12 = pd.read_csv('../meRIP-seq/hl60_delta_mtyl_table.txt',sep='\t').rename(columns={'name':'gene_name'})
    M = get_intersect_df(genes,mtyl_bed12)

    # RNA Methylation 
    out = M.reset_index(drop=True)
    return out 

### Write excel file 

In [17]:
# multi index
# out = 
# pd.DataFrame(
#     # [key.split(' & ') + [val] for key, value in all_genes.items() 
#      # for val in value if value]
# )#.rename({0:'1st',1:'2ed',2:'3ed',3:'gene_name'},axis=1)

sheet1 = make_comp_final_table(genes = all_genes_list,data = comps)
sheet2 = make_scrn_final_table(genes = all_genes_list,data = scrns)
sheet3 = make_mtyl_final_table(genes = all_genes_list)

In [18]:
# # multi index
# out = pd.DataFrame(
#     [key.split(' & ') + [val] for key, value in all_genes.items() 
#      for val in value if value]
# ).rename({0:'1st',1:'2ed',2:'3ed',3:'gene_name'},axis=1)

# sheet1 = make_comp_final_table(genes = all_genes_list,data = comps).loc[out.gene_name,:].set_index(pd.MultiIndex.from_frame(out))
# sheet2 = make_scrn_final_table(genes = all_genes_list,data = scrns).loc[out.gene_name,:].set_index(pd.MultiIndex.from_frame(out))
# sheet3 = make_mtyl_final_table(genes = all_genes_list)

In [19]:
# intersections
writer = pd.ExcelWriter(f'Target-genes/differential-analysis-values.xlsx', engine='xlsxwriter')

sheet1.to_excel(writer, sheet_name='Comparisons')
sheet2.to_excel(writer, sheet_name='CRISPRi-screens')
sheet3.to_excel(writer, sheet_name='Methylation',index=False)

writer.save()

### Print gene names

In [20]:
# for key, value in all_genes.items():
#     if value: 
#         print(key)
#         for val in value: 
#             print(f'\t{val}')
#         print('_'*30)

In [21]:
!date

Wed Mar 30 11:19:53 PDT 2022
